# Model strategy

Notebook to compare model strategies, including clinic or not in the model and the effect on the predictions

## Load packages

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from sklearn.ensemble import HistGradientBoostingClassifier

## Load data

In [ ]:
featuretable = pd.read_parquet("../data/processed/featuretable.parquet")

featuretable["no_show"] = (
    featuretable["no_show"].replace({"no_show": "1", "show": "0"}).astype(int)
)
featuretable["hour"] = featuretable["hour"].astype("category")
featuretable["weekday"] = featuretable["weekday"].astype("category")

print(featuretable.dtypes)

X, y = featuretable.drop(columns="no_show"), featuretable["no_show"]

In [ ]:
lgboost_model = HistGradientBoostingClassifier(
    learning_rate=0.05,
    max_iter=300,
    categorical_features=["hour", "weekday"],
)

In [ ]:
fitted_model = lgboost_model.fit(X, y)
preds = fitted_model.predict_proba(X)
preds

In [ ]:
featuretable_preds = featuretable.copy()
featuretable_preds["preds"] = preds[:, 1]

In [ ]:
featuretable_preds["clinic"] = featuretable_preds.index.get_level_values("poli")
featuretable.plot.hist(by="clinic", y="preds")